### Exploring social ties between GitHub collaborators and resulting repository popularity

Tracy Lo</br>
Student #: 998963028

#### Introduction

The research topic that has been chosen is a study on the social ties between GitHub collaborators and the resulting popularity of the projects where the collaborators have worked together in the past. In open source communities, there are various factors which may motivate one to participate in a project team – of these motivations, social ties and relationships are amongst the top reasons for project collaboration in open source projects. An investigation on the social ties between collaborators on Github and the impacts on the resulting repository popularity shall be explored.

#### Revised Project Plan 

The project plan for the remaining time period of the project has been revised based on the feedback gathered from V1:

* Week 2 (V2):
  * Create graphs - 6 hrs
    * Developer-project network - 3 hrs 
    * Developer-developer network - 3 hrs
  * Popularity Prediction - 4 hrs
    * K-Means Implementation
    * Model Evaluation (Silhouette)
* Week 3 (V3): 
  * Make revisions - 2 hrs
  * Full validation of data preparation - 4 hrs
  * Evaluate results and review process - 4 hrs
* Week 4:
  * Create Final Report - 5 hrs

During each phase, a base of 10 hours will be allocated for tasks. Week 4 will require less time (5 hrs) as the work should be complete, save for formatting of the final report. Additional buffer time (~2 hrs) will be added to each version in the event that any issues or unexpected changes that may arise.

#### Project Plan 

The project plan has been constructed following the tasks outlined in CRISP-DM and broken down into a four-week plan:

* Week 1 (V1):
  * Data Understanding: exploration, verification - 3.5 hrs
    * Ingest data - 0.5 hr
    * Describe and assess metadata - 3 hrs
  * Data Preparation: selection, cleaning - 4.5 hrs
    * Cleaning - 1.5 hr
    * Transformations on Big Query - 3 hrs
  * Create project plan - 2 hr
    * Set up Databricks notebook - 1 hr
    * Task breakdown and scheduling - 1 hr
* Week 2 (V2):
  * Make updates to project summary and plan based on instructor feedback - 6 hrs 
    * Tasks to be determined depending on feedback received
  * Full validation of data preparation - 4 hrs
* Week 3 (V3): 
  * Make revisions - 2 hrs
  * Modelling: Build and assess model - 5 hrs
    * Create graph to model developer social ties - 2.5 hrs
    * Implement k-means algorithm on repository popularity - 2.5 hrs
  * Evaluate results and review process - 3 hrs
* Week 4:
  * Create Final Report - 5 hrs

During each phase, a base of 10 hours will be allocated for tasks. Week 4 will require less time (5 hrs) as the work should be complete, save for formatting of the final report. Additional buffer time (~2 hrs) will be added to each version in the event that any issues or unexpected changes that may arise.

#### Data Understanding

##### Ingestion and Description of Metadata 
As part of the data understanding process, the data was ingested from GHTorrent on Google BigQuery. Once ingested, the individual tables were assessed for structure, granularity, accuracy, temporality, and scope.

The following tables from GHTorrent [6] on Google BigQuery were ingested. Data columns which are required by the scope of the project topic are bolded.
* [ghtorrent-bq:ght.users]: This table describes all users on GitHub.
  * **id**: user id
  * **login**: user login
  * company: if user belongs to a company, name of company - not required
  * **created_at**: user's date of creation
  * **type**: user or org (org = organizations)
  * **fake**: boolean (fake users are users who appear as authors or commiters of commits)
  * **deleted**: boolean (deleted users are users which GHTorrent cannot get details for)
  * long: longitude value for location of user - not required
  * lat: latitude value for location of user - not required
  * country_code: user's country of origin - not required
  * state: user's state of origin - not required
  * city: user's city of origin - not required
  * location: user's location of origin - not required
* [ghtorrent-bq:ght.projects]: This table describes all projects on GitHub.
  * **id**: project id
  * url: project url - not required
  * **owner_id**: user id of project owner
  * **name**: project name
  * description: project description - not required
  * language: project language - not required
  * **created_at**: date which project was created at
  * forked_from: if project was forked from another project, o
  * **deleted**: boolean (whether project is deleted from GitHub)
  * **updated_at**: date which project was updated at
* [ghtorrent-bq:ght.project_members]: This table describes all members (defined as users who have commit access) of specific projects.
  * **repo_id**: project id
  * **user_id**: user id
  * **created_at**: date which user joined project; only accurate if GHTorrent has recorded the event else the date is replaced with the creation date of the user or project
  * dont_use: no description in GHTorrent schema - not required
* [ghtorrent-bq:ght.commits]: This table describes all unique commits made.
  * **id**: commit id
  * sha: global commit id - not required
  * author_id: commit author id - not required
  * **committer_id**: commiter id
  * **project_id**: project which commit was associated with; may differ from project id that commit was initially pushed to (requires project_commits)
  * **created_at**: date which commit was made
* [ghtorrent-bq:ght.project_commits]: This table describes all commits made for specific projects.
  * **project_id**: project id 
  * **commit_id**: commit id 
* [ghtorrent-bq:ght.watchers]: This table describes all users who have watched (starred) projects.
  * **repo_id**: project id
  * **user_id**: user id
  * **created_at**: date which user watched/starred project; only accurate if GHTorrent has recorded the event else the date is replaced with the creation date of the user or project
* [ghtorrent-bq:ght_2018_04_01.pull_request]: This table describes all pull requests made.
  * **id**: pull request id
  * head_repo_id: repo id that pull request is from
  * **base_repo_id**: repo id that pull request goes to
  * head_commit_id: commit id that pull request is from
  * base_commit_id: commiit id that pull request goes to
  * **pullreq_id**: pull request id on Github
  * **intra_branch**: boolean (signifies whether head and base repos are the same)
* [ghtorrent-bq:ght_2018_04_01.pull_request_history]: This table describes all pull requests and their corresponding events.
  * **id**: pull request id
  * **pull_request_id**: pull request id on Github
  * **created_at**: date which pull request was made
  * **action**: action of event corresponding to entry (can be opened/closed/merged/reopened/synchronized)
  * **actor_id**: user id of user who initiated pull request

#### Data Preparation

##### Transformations (on BigQuery)
The tables above were transformed on BigQuery through intrarecord and interrecord structuring to create more meaningful tables for the purpose of the project topic.
* Active Projects
  * Only active projects will be considered for the scope of the project. According to Gousios [6], GHTorrent returns all projects, including those which have been deleted. For the purposes of the project, these deleted projects were filtered out. In addition, it was decided that the definition of "active projects" included only projects which had:
    * commits within the last year -  this was achieved through the creation of a new "active projects - commits" table which utilized columns from the commits, projects, and project_commits tables, filtered by the project created_at field.
    * pull requests within the last year - this was achieved through the creation of a new "active projects - pull requests" table which utilized columns from the pull_requests, projects, and pull_request_history tables, filtered by the project created_at field.
* Project Members
  * Antwerp et al. [3] creates a graph to model the developer network using a list of projects and the respective developers working on said projects. To determine the members of a specific project, specific columns chosen above are combined from the users, projects and project_members table to create a list of projects with the respective project owners and members. Users which are "fake" or "deleted" were removed from the dataset.
* Project Stars on Weekly Basis
  * To determine the popularity of a project, Borges et al. [4] investigates project popularity through the usage of the “star” functionality. According to Gousios [6], the watchers table provides the list of users who have starrted/watched a project. The watchers table was transformed such that each star action was grouped into the respective projects and assigned a created_at week number. The total number of star actions were then summed to retrieve a weekly count of each project's stars.  
  
##### Cleaning
* Formatting of all created_at columns is YYYY-MM-DD HH:MI:SS.SSSSSSSS UTC, which does not adhere with the correct timestamp format (YYYY-MM-DD HH:MI:SS) and therefore will need to be modified
* Invalid values may occur for:
  * created_at timestamps, which is accurate only if GHTorrent has recorded the event - else the date is replaced with the creation date of the user or project. Comparisons between project and member created_at dates will be compared to watchers and project_members created_at dates - if they are the same value, possibly exclude them from the dataset. [7]
* No missing or null values were found in the transformed tables

In [7]:
%scala

// Import required libraries
import java.sql.Timestamp
import org.apache.commons.io.IOUtils
import java.net.URL
import java.nio.charset.Charset

//Patching string class with new functions that have a default value if conversion to another type fails
implicit class StringConversion(val s: String) {
def toTypeOrElse[T](convert: String=>T, defaultVal: T) = try {
    convert(s)
  } catch {
    case _: Throwable => defaultVal
  }
  
  def toIntOrElse(defaultVal: Int = 0) = toTypeOrElse[Int](_.toInt, defaultVal)
  def toDoubleOrElse(defaultVal: Double = 0D) = toTypeOrElse[Double](_.toDouble, defaultVal)
  def toDateOrElse(defaultVal: java.sql.Timestamp = java.sql.Timestamp.valueOf("1970-01-01 00:00:00")) = toTypeOrElse[java.sql.Timestamp](java.sql.Timestamp.valueOf(_), defaultVal)
}

// Load CSV files
val activeprojects_prRDD = sc.parallelize( IOUtils.toString( new URL("https://drive.google.com/uc?export=download&id=1-CqOeSGp98VHOXQhS3oGOOq19fIBUELw"), Charset.forName("utf8")).split("\n")).map(line => line.split(",", -1).map(_.trim)).filter(line => line(0) != "project_id")

val activeprojects_commitsRDD = sc.parallelize( IOUtils.toString( new URL("https://drive.google.com/uc?export=download&id=1RHb6yDmbdwEXFV1gOCZEwCM0R45YhzVD"), Charset.forName("utf8")).split("\n")).map(line => line.split(",", -1).map(_.trim)).filter(line => line(0) != "project_id")

val activeprojects_allRDD = sc.parallelize( IOUtils.toString( new URL("https://drive.google.com/uc?export=download&id=1RWNY18NUPZqHey2cMkF4_4OSxciNXYYy"), Charset.forName("utf8")).split("\n")).map(line => line.split(",", -1).map(_.trim)).filter(line => line(0) != "project_id")

val duration_commitsRDD = sc.parallelize( IOUtils.toString( new URL("https://drive.google.com/uc?export=download&id=1RU7SKoaPh5ahupzvtdBHhPRdvSzsz17x"), Charset.forName("utf8")).split("\n")).map(line => line.split(",", -1).map(_.trim)).filter(line => line(0) != "repo_id")

val duration_prRDD = sc.parallelize( IOUtils.toString( new URL("https://drive.google.com/uc?export=download&id=1XE6idJDlC0jdpQjgxTkzGbVe0ku36C8x"), Charset.forName("utf8")).split("\n")).map(line => line.split(",", -1).map(_.trim)).filter(line => line(0) != "repo_id")

val watchersRDD = sc.parallelize( IOUtils.toString( new URL("https://drive.google.com/uc?export=download&id=12hkHE4MuKIJKO3e4TBbkTptk_sfit6wr"), Charset.forName("utf8")).split("\n")).map(line => line.split(",", -1).map(_.trim)).filter(line => line(0) != "repo_id")

val topproj_watchersRDD = sc.parallelize( IOUtils.toString( new URL("https://drive.google.com/uc?export=download&id=1gSR25-CDD8wBidcgU3Gx1PvxAXlaQUfS"), Charset.forName("utf8")).split("\n")).map(line => line.split(",", -1).map(_.trim)).filter(line => line(0) != "repo_id")

//val allwatchersRDD = sc.parallelize( IOUtils.toString( new URL("https://drive.google.com/uc?export=download&id=1dhNivmsOrKTUPUpDQXQf0UU78L7pXS5d"), Charset.forName("utf8")).split("\n")).map(line => line.split(",", -1).map(_.trim)).filter(line => line(0) != "repo_id")

val projectsdurationRDD = sc.parallelize( IOUtils.toString( new URL("https://drive.google.com/uc?export=download&id=1WPL-Bv1qZR_EB4eLvtBZCHTmke3DWQyi"), Charset.forName("utf8")).split("\n")).map(line => line.split(",", -1).map(_.trim)).filter(line => line(0) != "project_id")

#### SQL Queries (Run on BigQuery)

According to Rattenbury et al. [7], data should be assessed for whether or not it is "stale" as part of the temporality assessment. In terms of Github, "stale" data can be represented by repositories which are inactive (no "activity" defined by commits or pull requests). In a study on mining Github data, Kalliamvakou et. al. [8] identified that most projects are inactive on Github. In order to ensure that the dataset used in the project will create meaningful results, the following SQL query was run on BigQuery to find all active projects with commits within the last year (entire query results in the "[reliable-stage-199320:github.projects_activecommits_1yr]" table which is imported as activeprojects_commitsRDD). This was done by joining the projects_commits and commits table to find the commits associated with each project. The resulting table was then joined with the projects table to get more details (i.e. project name, project owner id, project creation date, project updated date) of the projects which had commits to it. The final table was then filtered to show only projects which had commits created within the past year in order to show "active" projects. However, as the table displays all commits made to a project within the past year, users which made multiple commits would appear multiple times. As part of the data scope assessment, a new column of "ranking" was created to sort all commits by their date in descending order. In order to get distinct users who made commits to the projects within the past year, the "ranking" column was populated using window functions, were used to rank the commits by the commit date and only the most recent commit which a user made was taken.

In [10]:
'''
Active Projects with Commits in the Past Year: following query resulted in the [reliable-stage-199320:github.projects_activecommits_1yr] table

SELECT
  project_id,
  project_owner_id,
  project_name,
  project_created_date,
  last_commit_date,
  ranking
FROM (
  SELECT
    c.project_id AS project_id,
    p.owner_id AS project_owner_id,
    p.name AS project_name,
    p.created_at AS project_created_date,
    c.created_at AS last_commit_date,
    ROW_NUMBER() OVER (PARTITION BY project_id ORDER BY last_commit_date DESC) AS ranking
  FROM
    [ghtorrent-bq.ght.commits] AS c
  JOIN
    [ghtorrent-bq.ght.projects] AS p
  ON
    p.id = c.project_id
  WHERE
    (c.created_at BETWEEN DATE_ADD(CURRENT_TIMESTAMP(), -1, 'YEAR')
      AND CURRENT_TIMESTAMP()) )
WHERE
  ranking = 1
ORDER BY
  last_commit_date desc
'''

Kalliamvakou et. al. [8] study on mining Github data also identified that most projects have very little commits which meant that the first table which returned active projects based soley on commits would not be enough. Thus, pull requests were also incorporated to create a full list of active projects (entire query results in the "[reliable-stage-199320:github.projects_activepr_1yr]" table which is imported as activeprojects_prRDD). Active projects with pull requests within the past year were found by joining the pull_request and pull_request_history tables to determine the pull requests associated with each project. The resulting table was then joined with the projects table to get more details (i.e. project name, project owner id, project creation date, project updated date) of the projects which had pull requests to it. The final table was then filtered to show only projects which had pull requests created within the past year in order to show "active" projects. However, as the table displays all pull requests made to a project within the past year, users which made multiple pull requests would appear multiple times. As part of the data scope assessment, a new column of "ranking" was created to sort all PRs by their date in descending order. In order to get distinct users who made PRs to the projects within the past year, the "ranking" column was populated using window functions, were used to rank the PRs by the PR date and only the most recent PR which a user made was taken.

In [12]:
'''
Active Projects with Pull Requests in the Past Year: following query resulted in the [reliable-stage-199320:github.projects_activepr_1yr] table.

SELECT
  project_id,
  project_name,
  project_created_date,
  last_pr_date,
  ranking
FROM (
  SELECT
    pr_prh.pullreq_id AS pullreq_id,
    p.owner_id AS project_owner_id,
    p.id as project_id,
    p.name AS project_name,
    p.created_at AS project_created_date,
    pr_prh.created_at AS last_pr_date,
    ROW_NUMBER() OVER (PARTITION BY p.id ORDER BY last_pr_date DESC) AS ranking
  FROM
    [ghtorrent-bq.ght.projects] AS p
  JOIN (
    SELECT
      pr.id AS id,
      pr.base_repo_id AS base_repo_id,
      pr.pullreq_id AS pullreq_id,
      prh.pull_request_id AS pull_request_id,
      prh.action AS action,
      prh.created_at AS created_at
    FROM
      [ghtorrent-bq.ght_2018_04_01.pull_request_history] AS prh
    JOIN
      [ghtorrent-bq.ght_2018_04_01.pull_requests] AS pr
    ON
      prh.pull_request_id = pr.id
    WHERE
      action = 'merged' and (prh.created_at BETWEEN DATE_ADD(CURRENT_TIMESTAMP(), -1, 'YEAR')
      AND CURRENT_TIMESTAMP()) ) AS pr_prh
  ON
    p.id = pr_prh.base_repo_id)
WHERE
  ranking = 1
ORDER BY
  last_pr_date DESC
'''

Once the list of projects which had commits and pull requests made within the past year were retrieved, a new table "[reliable-stage-199320:active_projects_commitsprscombined]" (imported as activeprojects_allRDD) was created to combine the projects from both lists.

In [14]:
'''
All active projects (pull requests and commits within past year): following query resulted in the [reliable-stage-199320:active_projects_commitsprscombined] table.

SELECT 
  project_id, 
  project_name, 
  project_created_date 
FROM 
  [reliable-stage-199320:github.projects_activepr_1yr], 
  [reliable-stage-199320:github.projects_activecommits_1yr]
'''

Antwerp et. al. [8] study on open source developer relationships began with gathering the required data to construct a developer-developer network. In order to construct the network, a list of projects and the developers who worked on said projects were required. In addition, the dates of each developer's first and last contribution to the project were required. This would be used to calculate whether or not developers collaborated - if their first and last contribution date fell within another developer's first and last contribution date, then the two would have worked together at some point. To gather this data in the Github dataset, the commits and pull requests were looked at separately. 

To begin, the commits and users tables were joined to determine the user details of each committer and the first and last commit the particular user made (using max and min on window functions) within the past year. From there, the resulting table was joined with the list of active projects ("[reliable-stage-199320:active_projects_commitsprscombined]" table above) in order to determine which users had made commits to the list of active projects which were identified in the previous step. The final "[reliable-stage-199320:github.usercommits_activeprojects]" table query is shown below, and is imported as duration_commitsRDD.

In [16]:
'''
All users who made commits (with first and last commit dates) on active projects: following query resulted in the [reliable-stage-199320:github.usercommits_activeprojects] table.

SELECT
  ap.project_id AS project_id,
  uc.member_id AS member_id,
  uc.member_login AS member_login,
  uc.latest_commit AS latest_commit,
  uc.oldest_commit AS oldest_commit
FROM (
  SELECT
    u.id AS member_id,
    u.login AS member_login,
    c.project_id AS project_id,
    c.created_at,
    MAX(c.created_at) OVER (PARTITION BY project_id, member_id, member_login) AS latest_commit,
    MIN(c.created_at) OVER (PARTITION BY project_id, member_id, member_login) AS oldest_commit
  FROM
    [ghtorrent-bq:ght.commits] AS c
  JOIN
    [ghtorrent-bq:ght.users] AS u
  ON
    c.author_id = u.id
  WHERE
    u.fake = FALSE
    AND u.deleted = FALSE
    AND (c.created_at BETWEEN DATE_ADD(CURRENT_TIMESTAMP(), -1, 'YEAR')
      AND CURRENT_TIMESTAMP())
  GROUP BY
    member_id,
    member_login,
    project_id,
    c.created_at
  ORDER BY
    project_id) AS uc
JOIN
  [reliable-stage-199320:github.active_projects_commitsprscombined] AS ap
ON
  ap.project_id = uc.project_id
GROUP BY 
  project_id, 
  member_id, 
  member_login, 
  latest_commit, 
  oldest_commit
'''

Next, the pull requests, pull requests history and users tables were joined to determine the user details of each pull request author and the first and last pull request the particular user made (using max and min on window functions) within the past year. From there, the resulting table was joined with the list of active projects ("[reliable-stage-199320:active_projects_commitsprscombined]" table above) in order to determine which users had made pull requests to the list of active projects which were identified in the previous step. The final "[reliable-stage-199320:github.userpr_activeprojects]" table query is shown below, and is imported as duration_prRDD.

In [18]:
'''
All users who made pull requests (with first and last pull request dates) on active projects:  Following query resulted in the [reliable-stage-199320:github.userpr_activeprojects] table.

SELECT
  ap.project_id AS project_id,
  upr.member_id AS member_id,
  upr.member_login AS member_login,
  upr.latest_pull_request AS latest_pr,
  upr.oldest_pull_request AS oldest_pr
FROM (
  SELECT
    u.id AS member_id,
    u.login AS member_login,
    pr_prh.base_repo_id AS project_id,
    pr_prh.created_at,
    MAX(pr_prh.created_at) OVER (PARTITION BY project_id, member_id, member_login) AS latest_pull_request,
    MIN(pr_prh.created_at) OVER (PARTITION BY project_id, member_id, member_login) AS oldest_pull_request
  FROM (
    SELECT
      pr.id AS id,
      pr.base_repo_id AS base_repo_id,
      pr.pullreq_id AS pullreq_id,
      prh.pull_request_id AS pull_request_id,
      prh.action AS action,
      prh.actor_id as actor_id,
      prh.created_at AS created_at
    FROM
      [ghtorrent-bq:ght_2018_04_01.pull_request_history] AS prh
    JOIN
      [ghtorrent-bq:ght_2018_04_01.pull_requests] AS pr
    ON
      prh.pull_request_id = pr.id) as pr_prh
  JOIN
    [ghtorrent-bq:ght.users] AS u
  ON
    pr_prh.actor_id = u.id
  WHERE
    u.fake = FALSE
    AND u.deleted = FALSE
    AND (pr_prh.created_at BETWEEN DATE_ADD(CURRENT_TIMESTAMP(), -1, 'YEAR')
      AND CURRENT_TIMESTAMP())
  GROUP BY
    member_id,
    member_login,
    project_id,
    pr_prh.created_at
) AS upr
JOIN
  [reliable-stage-199320:github.active_projects_commitsprscombined] AS ap
ON
  ap.project_id = upr.project_id
GROUP BY
  project_id,
  member_id,
  member_login,
  latest_pr,
  oldest_pr
'''

The separate tables of users who have created commits and pull requests are then combined in order to get the full list of users who have create commits, pull requests, or both to any of the active projects (projects which have pull requests or commits within the past year). The resulting table query is shown below and is imported as projectsdurationRDD.

In [20]:
'''
Following query resulted in table to display list of all members and their latest/oldest commit or PR within active projects (commits or PRs within last year)

SELECT
  project_id,
  member_id,
  member_login,
  latest_pr,
  oldest_pr,
  latest_commit,
  oldest_commit
FROM
  [reliable-stage-199320:github.userpr_activeprojects],
  [reliable-stage-199320:github.usercommits_activeprojects]
ORDER BY
  project_id'''

In order to perform the project popularity analysis, Borges et. al [4] requires the number of stars acquired per project per week for a 52 week period. The following query shown below was run to gather the number of stars acquired on a weekly basis. To begin, the watchers table which lists all possible projects is joined with the "[reliable-stage-199320:github.active_projects_commitsprscombined]" table to return the list of active projects based on the last commit or PR within the past year. Next, the week function was used to extract the week from the created_at dates that stored the date of the star event. Next, all star events for a project within the particular week of the year were summed in order to get the total number of star events for a given week. The resulting table is imported as watchersweekly.

In [22]:
'''
Watchers per Week - ran on BigQuery

SELECT
  t.repo_id AS repo_id,
  t.repo_name as repo_name,
  COUNT(*) AS number_watchers,
  t.week_number AS week_number
FROM (
  SELECT
    w.repo_id as repo_id,
    ap.project_name as repo_name,
    w.created_at as created_at,
    WEEK(w.created_at) AS week_number
  FROM
    [ghtorrent-bq:ght.watchers] as w
    join
    [reliable-stage-199320:github.active_projects_commitsprscombined] as ap
    on w.repo_id = ap.project_id
  ORDER BY
    repo_id,
    repo_name,
    week_number
  ) AS t
GROUP BY
  repo_id,
  repo_name,
  week_number
ORDER BY
  repo_id,
  repo_name,
  week_number
'''

#### activeprojects_prRDD - Exploratory Analysis

To begin the exploratory analysis portion, the csv files were loaded in a previous step. Next, the csv files were cleaned according to the data preparation process as mentioned - as part of the data accuracy assessment, date times should be adjusted to a standard time zone and in the correct format. The dataset was originally in UTC format already; however, the timestamps all have a "UTC" string at the end and therefore will be removed in the cleanActiveProjectsPR function.

In [24]:
%scala

// Define schema
case class activeProjectsPR(                 
   project_id: Int,
   project_name: String,
   project_created_at: java.sql.Timestamp,
   last_pr_date: java.sql.Timestamp,
   ranking: Int
  )

// Map data to schema
def cleanActiveProjectsPR(row:Array[String]):activeProjectsPR = {
return activeProjectsPR(
    row(0).toIntOrElse(),
    row(1),
    Timestamp.valueOf(row(2).replace("UTC", "")),
    Timestamp.valueOf(row(3).replace("UTC", "")),
    row(4).toIntOrElse()
  )
}

// Create sql table 
val activeprojectspr = activeprojects_prRDD.map(r => cleanActiveProjectsPR(r)).toDF()
// register this data as an SQL table
activeprojectspr.createOrReplaceTempView("activeprojectspr")

In [25]:
# Place data in dataframe and observe
df_pr = sqlContext.table("activeprojectspr")
display(df_pr)

As part of the data temporality assessment, the dates of which the last pull request of each project was observed. From BigQuery, the table was from April 1 2018, which can be taken as the date that the dataset was collected. The results below show that the records temporal range is a very small window, between March 28 - March 31 2018. The data can be considered "fresh" (as opposed to "stale") since the last PR date (which can be used to represent the last time that the project has been modified) is within the year limit which was set as an "active" project requirement.

In [27]:
%sql
/* A count of active projects in each month shows that all PRs within the 3rd month (March). */

SELECT month(last_pr_date) as month_of_last_pr,
       count(project_id) as active_projects   
FROM activeprojectspr
GROUP BY month(last_pr_date)
ORDER BY month_of_last_pr


In [28]:
%sql
/* A count of active projects for each day shows that the PRs are all within the last 4 days of the month (28th - 31st). */

SELECT day(last_pr_date) as day_of_last_pr,
       count(project_id) as active_projects   
FROM activeprojectspr
GROUP BY day(last_pr_date)
ORDER BY day_of_last_pr


#### activeprojects_commitsRDD - Exploratory Analysis

To begin the exploratory analysis portion, the csv files were loaded in a previous step. Next, the csv files were cleaned according to the data preparation process as mentioned - as part of the data accuracy assessment, date times should be adjusted to a standard time zone and in the correct format. The dataset was originally in UTC format already; however, the timestamps all have a "UTC" string at the end and therefore will be removed in the cleanActiveProjectsCommits function.

In [30]:
%scala

// Define schema
case class activeProjectsCommits(                 
   project_id: Int,
   project_name: String,
   project_owner_id: Int,
   project_owner_login: String,
   project_created_at: java.sql.Timestamp,
   last_commit_date: java.sql.Timestamp,
   ranking: Int
  )

// Map data to schema
def cleanActiveProjectsCommits(row:Array[String]):activeProjectsCommits = {
return activeProjectsCommits(
    row(0).toIntOrElse(),
    row(1),
    row(2).toIntOrElse(),
    row(3),
    Timestamp.valueOf(row(4).replace("UTC", "")),
    Timestamp.valueOf(row(5).replace("UTC", "")),
    row(6).toIntOrElse()
  )
}

// Create sql table 
val activeprojectscommits = activeprojects_commitsRDD.map(r => cleanActiveProjectsCommits(r)).toDF()
// register this data as an SQL table
activeprojectscommits.createOrReplaceTempView("activeprojectscommits")

In [31]:
# Place data in dataframe and observe
df_commits = sqlContext.table("activeprojectscommits")
display(df_commits)

As part of the data temporality assessment, the dates of which the last commit of each project was observed. The results below show that the records temporal range is a very large, spanning the entire year. This is a sharp contrast with the pull requests dataset, which was concentrated between the 4 days of April 28-31st 2018. However, the data can be still considered "fresh" (as opposed to "stale") since the last PR date (which can be used to represent the last time that the project has been modified) is within the year limit which was set as an "active" project requirement.

In [33]:
%sql
/* A count of active projects in each month for 12 months shows that active projects are evenly distributed throughout the 1-year period. */

SELECT month(last_commit_date) as month_of_last_commit,
       count(project_id) as active_projects   
FROM activeprojectscommits
GROUP BY month(last_commit_date)
ORDER BY month_of_last_commit


As part of the data exploration process, data scope can be evaluated by observing the characteristics of the dataset. In the cell below, an analysis will be performed on the activeprojectcommits dataset in order to determine the distribution of projects a user owns. As shown in the graph, the majority of users (93%) own 1 project, with a small percentage (7%) of users owning 2 projects.

In [35]:
%sql
/* Distribution of projects a user owns. Majority of users own 1 project, with 5% of users owning 2 projects. */

SELECT project_owner_id, 
       count(distinct(project_id)) as projects
FROM activeprojectscommits
GROUP BY project_owner_id

#### Watchers/Starred Events - Exploratory Analysis

To begin the exploratory analysis portion, the csv files were loaded in a previous step. Next, the csv files were cleaned and imported into a dataframe.

In [37]:
%scala

// Define schema
case class watchersWeekly(                 
   repo_id: Int,
   repo_name: String,
   number_watchers: Int, 
   week_number: Int
  )

// Map data to schema
def cleanWatchersWeekly(row:Array[String]):watchersWeekly = {
return watchersWeekly(
    row(0).toIntOrElse(),
    row(1),
    row(2).toIntOrElse(),
    row(3).toIntOrElse()
  )
}

// Create sql table 
val watchersweekly = watchersRDD.map(r => cleanWatchersWeekly(r)).toDF()
// register this data as an SQL table
watchersweekly.createOrReplaceTempView("watchersweekly")

In [38]:
# Place data in dataframe and observe
df = sqlContext.table("watchersweekly")
display(df)

In order to perform the popularity analysis using the star events of a project, Borges et al. [4] only considers the stars gained in the last 52 weeks of each repository and removes all repositories with less than 52 weeks. First, a count of the total number of weeks each repository has is performed in the first query. As shown, there are repositories which have less than 53 weeks.

In [40]:
%sql
/* Find the number of weeks each project has star records for */

select t.repo_id, t.repo_id, t.total_weeks from (select repo_id, repo_name, count(week_number) as total_weeks
from watchersweekly
group by repo_id, repo_name) as t

This query follows the assessment criteria for data scope, which includes the observation of consistency of record fields. Since there are inconsistencies in the record fields (some repositories have 53 weeks worth of star counts; other have less), it is important to remove the inconsistent records (repositories with less than 53 weeks worth of data) to retain a consistent dataset. The next query shown below is performed to remove all repositories which have less than 53 weeks of star events recorded.

In [42]:
%sql
/* Find all projects which have star records for 53 weeks (full year record) */

select t.repo_id, t.repo_id, t.total_weeks from (select repo_id, repo_name, count(week_number) as total_weeks
from watchersweekly
group by repo_id, repo_name) as t
where t.total_weeks = 53

Once the list of repositories which have consistent records of star events are retrieved, the counts of each week's star events are retrieved by joining the results of said table with the watchersweekly table, as shown below. A line plot displaying each project's total star events (y-axis) for each week of the year (x-axis) is shown below the query. It is observed that there are several repositories which have spikes in the total star events - including those with repo_id of 37, 528, and 540.

In [44]:
%sql
/* Find all projects which have star records for 53 weeks (full year record) */

select w.repo_id, w.repo_name, w.number_watchers, w.week_number
from
watchersweekly as w
join (select t.repo_id, t.repo_id, t.total_weeks from (select repo_id, repo_name, count(week_number) as total_weeks
from watchersweekly
group by repo_id, repo_name) as t
where t.total_weeks = 53) as fr
on w.repo_id = fr.repo_id

In [45]:
%scala

// register this data as an SQL table
val fr_watchersDF = sqlContext.sql("select w.repo_id, w.repo_name, w.number_watchers, w.week_number from watchersweekly as w join (select t.repo_id, t.repo_id, t.total_weeks from (select repo_id, repo_name, count(week_number) as total_weeks from watchersweekly group by repo_id, repo_name) as t where t.total_weeks = 53) as fr on w.repo_id = fr.repo_id")
fr_watchersDF.createOrReplaceTempView("fr_watchers")

#### Project Members and their latest/oldest Commit or PR within Active Projects - Exploratory Analysis

The query which was run in order to return all project members and their latest/oldest commit or PR was imported as projectdurationRDD and shown in the previous steps. The data preparation for the projectsdurationRDD including cleaning and parsing the data into a dataframe are performed in the following cell. Since the projects do not haveall fields for latest_pr, oldest_pr, latest_commit, and oldest_commit populated, these columns were first parsed as strings, then converted to timestamps, as shown below. The resulting dataframe "activeprojectsduration_temp" is shown below.

In [47]:
%scala

import org.apache.spark.sql.functions.{col, unix_timestamp}

// Define schema
case class projectsDuration(                 
   project_id: Int,
   project_name: String,
   member_id: Int,
   member_login: String,
   latest_pr: String,
   oldest_pr: String,
   latest_commit: String,
   oldest_commit: String
  )

// Map data to schema
def cleanProjectsDuration(row:Array[String]):projectsDuration = {
return projectsDuration(
    row(0).toIntOrElse(),
    row(1),
    row(2).toIntOrElse(),
    row(3),
    row(4),
    row(5),
    row(6),
    row(7)
  )
}

// Create sql table 
val activeprojectsduration_initial = projectsdurationRDD.map(r => cleanProjectsDuration(r)).toDF()

val updatedDF = activeprojectsduration_initial.withColumn("latest_pr", unix_timestamp(col("latest_pr"), "yyyy-MM-dd HH:mm:ss").cast("timestamp"))
val updatedDF1 = updatedDF.withColumn("oldest_pr", unix_timestamp(col("oldest_pr"), "yyyy-MM-dd HH:mm:ss").cast("timestamp"))
val updatedDF2 = updatedDF1.withColumn("latest_commit", unix_timestamp(col("latest_commit"), "yyyy-MM-dd HH:mm:ss").cast("timestamp"))
val activeprojectsduration_temp = updatedDF2.withColumn("oldest_commit", unix_timestamp(col("oldest_commit"), "yyyy-MM-dd HH:mm:ss").cast("timestamp"))

activeprojectsduration_temp.show()

As part of the data scope assessment, new fields for the oldest contribution date (oldest_date) and newest contribution date (newest_date) can be deduced from the existing columns of "latest_pr", "oldest_pr", "latest_commit", and "oldest_commit". Since all projects have either a set of contribution dates for PRs or commits - but not both - it is possible to find the oldest and newest contribution date by using the when function to determine whether or not a field is null and populate it with the remaining field. This is shown in the cell below.

In [49]:
%scala

import org.apache.spark.sql.functions._

// find oldest_date and newest_date between commit and pr dates
val updatedDF3 = activeprojectsduration_temp.withColumn("oldest_date", when($"oldest_pr".isNull, $"oldest_commit").otherwise($"oldest_pr"))
val updatedDF4 = updatedDF3.withColumn("newest_date", when($"latest_pr".isNull, $"latest_commit").otherwise($"latest_pr"))

updatedDF4.show()

To continue with the data scope assessment, a new dataframe is then created by performing the following:
- Dropping the individual newest and oldest contribution dates for PRs and commits
- Finding all projects with 2 or more members making contributions - if a project only has 1 active developer, then they are not collaborating with anyone else and therefore there are no social ties to investigate. A count of the developers grouped by project_id is performed and a filter condition is placed to include only projects with counts greater than or equal to 2 members. Dataframe then drops the counts column as the data is irrelevant to the analysis in the project

In [51]:
%scala

// display dataframe with oldest_contribution_date and newest_contribution_date
val updatedDF5 = updatedDF4.selectExpr("project_id", "project_name", "member_id", "member_login", "oldest_date as oldest_contribution_date", "newest_date as newest_contribution_date")

// find all projects with greater than 2 members making contributions
val updatedDF6 = updatedDF5.groupBy(updatedDF("project_id")).count.withColumnRenamed("count", "n").filter("n >= 2")

// filter dataframe of all projects using list of projects that have 2+ members
val activeprojectsduration = updatedDF5.join(updatedDF6, "project_id").drop("n")

activeprojectsduration.show()

// register this data as an SQL table
activeprojectsduration.createOrReplaceTempView("activeprojectsduration")

In [52]:
# Place data in dataframe and observe
df = sqlContext.table("activeprojectsduration")
display(df)

##### Basic Questions to Assess Data Structure
1. **Do all records in the dataset contain the same fields?**
   According to the describe function above, all 222 records in the watchersweekly dataset have the repo_id, number_watchers, and week_number field.
2. **How can you access the same fields across records? By position? By name?**
   Using the name of the column will allow access of the same fields across records.
3. **How are the records delimited/separated in the dataset? Do you need sophisticated parsing logic to separate the records from one another?**
   Records are delimited using a comma ",". Sophisticated parsing logic is not required in order to separate the records.
4. **How are record fields encoded? Human readable strings? Binary numbers? Hash keys? Compressed? Enumerated codes?**
   The record fields are encoded using human readable strings.

##### Basic Questions to Assess Data Granularity
- **What kind of thing (person, object, relationship, event, etc.) do the records represent?** 
  The records represent the total weekly number of "events" - the number of users who have starred a repository in a given week.
- **Are the records homogeneous (represent the same kinds of things)? Or heterogeneous?** 
    The records are homogeneous - they all show the weekly count of star events (up to 52 weeks) for each repository.
- **What alternative interpretations of the records are there? For example, if the records appear to be customers, could they actually be all known contacts (only some of which are customers)?** 
  Alternative interpretations of the records could be that the events actually represent the "watch" event; however, according to Gousios [6], the watchers events can be used interchangably with the stars events.

##### Basic Questions to Assess Data Accuracy
- **For date times, are time zones included or adjusted into a standard time zone like UTC?** 
  Yes, timestamps are converted to UTC (see created_at fields). However, according to Gousios [6], projects or events which had null values for the created_at field were replaced with the latest date that the corresponding user or project has been created. 
- **What is the measurable distribution of inaccuracies?** Due to the fact that the timestamps have been prepopulated, it is difficult to determine whether the original values were null and therefore inaccurate.

For questions pertaining to addresses, names, email addresses, numeric items like phone numbers and UPC codes, and sales transactions - these data accuracies do not apply as the Github dataset that is required for the project analysis does not use these variables. Questions pertaining to data input by sensors or manual/human input are also not relevant to the Github dataset.

##### Basic Questions to Assess Data Temporality
- **When was the dataset collected?** 
  Dataset was collected from September 1 2017 (from ghtorrent-bq:ght_2017_09_01 on BigQuery)
- **Were all the records and record fields collected/measured at the same time? If not, is the temporal range significant?** Records were not measured at the same time. The temporal range is quite significant, with 5 years in between the earliest and latest timestamps in the created_at column.
- **Are the timestamps associated with collection of the data known and available (as a record field) or as associated metadata?** Yes - see created_at columns
- **Have some records or record field values been modified after the time of creation? Are the timestamps of these modifications available?** Yes - see updated_at columns
- **In what ways can you determine if the data is "stale"? For example, you might have purchased a marketing leads databased and want to verify the contact information for the people represented in the dataset. Is it sufficient to sample the records and manually verify the data? Can you automatically verify it by using third-party services?** Can determine whether or not data is stale by manually comparing the repositories in the dataset to the repositories available to Github. However, this cannot be automically verified using a third-party service.
- **If there are conflicting values in the data (i.e. multiple mailing addresses for a person), can you use timestamps to determine which value is "correct"** If there are conflicting values in the data, it may be difficult to use timestamps to determine which value is "correct" due to the fact that the created_at field is only filled in accurately for starrings for which GHTorrent has recorded a corresponding event - otherwise, it is filled in with the latest date that the corresponding user or project has been created.

##### Basic Questions to Assess Data Scope
- **Given the granularity of the dataset, what characteristics of the things (i.e. people, objects, relationships, events, etc.) represented by the records are captured by the record fields? What characteristics are not captured?**  Events (when a member stars a repository) and relationships (developers working on a particular repository) are represented by the record fields. 
- **Are the record fields consistent? For example, does the customer's age field make sense relative to the date-of-birth field? If the record corresponds to a purchase transaction, does the cost of the listed set of purchased items add up to the total transaction amount?**  Fields are consistent - created_at and updated_at 
- **For the analysis you want to perform, can you deduce or infer additional relevant characteristics from the ones that you want? For example, can you infer the demographics of the people in a household from partner and dependents record fields?** Can infer when developers are working together on the same project at the same time by looking at their first and last commit dates. If time frames overlap, then they are collaborators/working on the same project at the same time.
- **Are the same record fields available for all records? Are they accessible via the same specification (position, name, etc.)?** The same record fields are available for all records.

#### Modelling

The modelling section of the report will focus on implementation of two models - first, using Graph X to model the developer network; and second, using k-means clustering on the amount of stars a project has to determine the popularity of the projects.

#### Graph X Implementation

As outlined in the methodology by Antwerp et al. [3], two graphs will be created in order to determine the developer network. - a developer-project bipartite graph, and a developer-developer graph.

In [60]:
%scala 

// import required packages
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

import org.apache.spark.sql.functions._

import org.apache.spark.ml.feature.{OneHotEncoder, StringIndexer}
import org.apache.spark.ml.param.ParamMap

#### Developer-Project Bipartite Graph

According to Antwerp et al. [4], a developer-project bipartite graph was created first to determine the relationships between projects and the developers who work on said projects. The vertices will be represented by both developers and projects, and an edge will be created if a developer has worked on a project. However, due to the fact that data preprocessing can be used to generate the data required to create the developer-developer network, this approach will be taken instead of building a developer-project bipartite graph first, then creating a developer-developer graph from the bipartite graph. As a result, the cell below for the bipartite graph is just a reference.

In [62]:
%scala

class VertexProperty extends Serializable {}

case class ProjectProperty(val id: Int, val name: String) extends VertexProperty
case class MemberProperty(val id: Int, val name: String) extends VertexProperty

val project: RDD[(VertexId, VertexProperty)] = activeprojectsduration.rdd.map(row => (row(0).asInstanceOf[Number].longValue, row(1).asInstanceOf[VertexProperty]))
val member: RDD[(VertexId, VertexProperty)] = activeprojectsduration.rdd.map(row => (row(2).asInstanceOf[Number].longValue, row(3).asInstanceOf[VertexProperty]))
val vertX = VertexRDD(project ++ member)
val edgX: RDD[Edge[Int]] = activeprojectsduration.select("project_id", "member_id").rdd.map(row => Edge(row(0).asInstanceOf[Number].longValue, row(1).asInstanceOf[Number].longValue))

val bpgraph: Graph[VertexProperty, Int] = Graph(vertX, edgX)

####Developer-Developer Graph

Next, a developer-developer graph will be created to determine the ties between developers who have worked together. The vertices will be represented by developers, and an edge will be created if a developer has worked with another developer. The edges will also have an attribute which represents how many occurences a particular pair of developers have worked together. 

The process of creating the developer-developer graph begins with encoding the project_id and member_id columns with 'p' and 'm' respectively. This is to ensure that if a project_id has the same value as a member_id, they will not erroneously match together. The resulting dataframe with the encoded ID values are stored in "dfe".

In [64]:
%scala

// to prevent matching of the same value from project_id and member_id, project_id was encoded with 'p' and member_id was encoded with 'm' and placed in new dataframe
val dfe = activeprojectsduration.select(
  concat(lit("p"), $"project_id").as("project_id"),
  $"project_name",
  concat(lit("m"), $"member_id").as("member_id"),
  $"member_login",
  $"oldest_contribution_date",
  $"newest_contribution_date"
)

dfe.show()

Once the new dataframe with the encoded IDs are created, it is duplicated into two dataframes "df1" and "df2". The resulting "df1" and "df2" are joined with the condition that the project_ids are the same but the member_ids are not the same - this will result in the developer-developer pairs for each project but ensure that each developer-developer pair does not include the developer paired with themselves. Data scope assessment is then performed on the new dataframe "ddpairs_initial" to deduce a new column "worked_together". Based on whether or not the contribution date range of the 1st developer is within the contribution date range of the 2nd developer, the worked_together column will be populated with 1 (worked together on the same project within the same timeframe), or 0 (have not worked together on the same project within the same timeframe; in other words, worked on the same project BUT not within the same timeframe). 

In order to create edges in the developer-developer graph, the edges must have a srcId and a dstId corresponding to the source and destination vertex identifiers, as well as an attr member which stores the edge property. Thus, the dataframe is grouped by the developer-developer pairs with each developer being either a "srcId" or "dstId" and the worked_together column is summed for each pair of developers to determine the total number of times that the pair worked together (stored in the "attr" column). The final dataframe of the developer-developer pairs and the number of times they have worked together is shown in "dd".

In [66]:
%scala

val df1 = dfe.select("project_id", "project_name", "member_id", "member_login", "oldest_contribution_date", "newest_contribution_date")
val df2 = dfe.select("project_id", "project_name", "member_id", "member_login", "oldest_contribution_date", "newest_contribution_date")

// join condition
val joinCondition = when($"df1.member_id" =!= $"df2.member_id", $"df1.project_id" === $"df2.project_id")


// find developer-developer pairs and its occurence (attr)
val ddpairs_initial = df1.alias("df1")
  .join(df2.alias("df2"), joinCondition).drop($"df2.project_name")

val ddpairs = ddpairs_initial.withColumn("worked_together", when($"df1.oldest_contribution_date" < $"df2.newest_contribution_date" and $"df1.newest_contribution_date" >= $"df2.oldest_contribution_date", 1).otherwise(0))
  // sort "source" (developer 1) and "destination" (developer 2)
  .groupBy(
     greatest("df1.member_id", "df2.member_id").as("srcId"), 
     least("df1.member_id", "df2.member_id").as("dstId"))
  .agg(sum($"worked_together").as("attr"))


val dd = ddpairs.selectExpr("srcId",
                      "dstId",
                      "attr")
dd.show()

As mentioned previously, developer-developer pairs who have a 0 in the "worked_together" column will not result in an edge since they are not collaborators on a project (did not work on a project together at the same time). Therefore, they are filtered from the dataframe, which results in the new dataframe "dd_filtered".

In [68]:
%scala 

val dd_filtered = dd.filter("attr > 0")

dd_filtered.show()

The "srcId" and "distId" columns of the "dd_filtered" dataframe is converted to integer values in preparation for the creation of the edges in the developer-developer graph. As mentioned previously, the edges of the graph are created when there is a working relationship (worked on a project together within the same timeframe) between two developers.

In [70]:
%scala

//convert srcId and dstId to int
val indexer = new StringIndexer().setInputCol("col").fit(
   dd_filtered.select("srcId").toDF("col").union(dd_filtered.select("dstId").toDF("col"))
)

val result = Seq("srcId", "dstId").foldLeft(dd_filtered){
  (dd_filtered, col) => indexer
    .copy(new ParamMap()
      .put(indexer.inputCol, col)
      .put(indexer.outputCol, s"${col}_idx"))
    .transform(dd_filtered)
}

val resultf = result.selectExpr("cast(srcId_idx as int) srcId_idx", "cast(dstId_idx as int) dstId_idx", "cast(attr as int) attr")
resultf.show()

By calling the describe function on the "resultf" dataframe, it can be seen that within the developer-developer pairs, there was a minimum of 2 times which the pairs worked together; the most that a pair of developers worked together was 8 times.

In [72]:
%scala

resultf.describe().show()

The "activeprojectsduration" dataframe is filtered for distinct member IDs in order to return a list of members for the active projects. This resulting dataframe "dfdist" of the project members will be used to construct the nodes of the developer-developer graph.

In [74]:
%scala

val dfdist = activeprojectsduration.selectExpr("member_id", "member_login").distinct
dfdist.show()

As a note on data scope assessment, the dataset has a total of 7675 developers.

In [76]:
%scala

dfdist.describe().show()

The following cell describes the developer-developer graph implementation. The nodes are first constructed using the dfdist dataframe and stored in "distmember". The edges are then constructed using the resultf dataframe and stored in "edgeRDD". Finally, the graph is created by calling the graph function and passing in "distmember" and "edgeRDD", and stored in "ddgraph". The number of verticies (nodes) in the graph are found to be 12532, which represents the 12532 developers. The number of edges in the graph are found to be 13598, which are the amount of ties between the 12532 developers. The number of triplets represent the 13598 developer-to-developer relationships and the attr property (in this case, how many times they have collaborated together) between them.

In [78]:
%scala

// define edges and vertices of developer-developer graph
val distmember: RDD[(VertexId, String)] = dfdist.select("member_id", "member_login").rdd.map(row => (row(0).asInstanceOf[Number].longValue, row(1).asInstanceOf[String]))
distmember.take(1)

val edgeRDD = resultf.toDF("srcId", "dstId", "attr").as[Edge[Int]].rdd

// create developer-developer graph
val ddgraph = Graph(distmember, edgeRDD)
ddgraph.cache()

println("Total number of edges in the graph: " + ddgraph.numEdges)
//ddgraph.edges.take(10).foreach(println)
println("Total number of vertices in the graph: " + ddgraph.numVertices)

println("Total number of triplets in the graph: " + ddgraph.triplets.count)

A common aggregation task is computing the degree of each vertex: the number of edges adjacent to each vertex. In the case of this analysis, it would present the number of developers that a particular developer has worked with. As shown in the cell below, developer with srcIdx of 0 worked with 165 other developers.

In [80]:
%scala

val maxDegrees: (VertexId, Int) = ddgraph.degrees.reduce( (a,b) => if (a._2 > b._2) a else b)
println("Developer who worked with the highest number of developers: " + maxDegrees)


Two simple queries back on the "result" and "dfe" tables (shown below) shows that it is the developer by the login of "deerawan" who has collaborated with 165 other developers.

In [82]:
%scala

result.select("srcId_idx", "srcId", "dstId_idx", "dstId", "attr").where($"srcId_idx" === 0  or $"dstId_idx" === 0).show(1)

In [83]:
%scala 

dfe.select("member_id", "member_login").where($"member_id" === "m1183849").show()

Next, the connected components algorithm was used to identify each connected component of the developer-developer graph. This implementation labels each connected component of the developer-developer graph with the ID of the vertex with the lowest number. Identification of connected components is helpful due to the fact that it can be used to approximate clusters - from the results below, a value of 2 clusters could be appropriate. In undirected graphs such as the developer-developer graph, two vertices are connected if they have a path connecting them. In the case of a single vertex, such as the 7675 components with 1 vertex, it is still considered connected due to the reflexive property which means that any vertex is strongly connected to itself. The 1 connected component with 4857 vertices refers to a single connected component where there are 4857 developers connected within. It can be concluded that the resulting developer-developer graph is a connected graph - and since the graph is shown to be connected, it can be said that the developer-developer network is resilient.

In [85]:
%scala

import scala.collection.immutable.ListMap

// Find connected components. Connected components algorithm labels each connected component of the graph with the ID of its lowest-numbered vertex. In a social network, connected components can approximate clusters.
val ddgraphcc = ddgraph.connectedComponents()

val ccNumVertices = 
  (ddgraph.vertices.map(pair => (pair._2,1))
  .reduceByKey(_+_)  // count the number of vertices contained in each connected component (indexed by the smallest vertex index in the connected component)
  .map(pair => pair._2)) // only maintain the number of vertices counted
println("Number of Connected Components: " + ccNumVertices.count)

ListMap(ccNumVertices.countByValue().toSeq.sortBy(_._1):_*).foreach(line => println(line._2 + " connected component(s) with " + line._1 + " vertices"));

Next, page rank algorithm was performed in order to determine the importance of each vertex (developer) in the developer-developer graph. In this case, if a developer has many ties with other developers,they will be ranked highly. The following cell returns the top 10 developers who have the most ties and therefore are the most signficant in the sense that the projects they participated in could be impacted by their abundance of their collaborator network.

In [87]:
%scala

// page rank on developer-developer-graph to find top 10 developers who are significant
val ranks = ddgraph.pageRank(0.0001).vertices
ranks
  .join(distmember)
  .sortBy(_._2._1, ascending=false) // sort by the rank
  .take(10) // get the top 10
  .foreach(x => println(x._2._2))

The member IDs of top 10 developers with the greatest significance were used to filter the activeprojectsduration dataframe in order to determine which projects they had worked on. These projects will then be subject to a separate analysis on their project popularity in order to determine whether or not their popularity is indeed greater than the average project. The new dataframe "td_report" is created using the SQL query below.

In [89]:
%scala 

//val topdevelopers: Array[String] = Array(stof, juanriaza, wong2, frewsxcv, JakeWharton, haikusw, nathenharvey, jbr, tenderlove, mmp)

// find projects which the top developers have contributed to
val td_report = activeprojectsduration.select("*").where($"member_login" === "stof" or $"member_login" === "juanriaza" or $"member_login" === "wong2" or $"member_login" === "frewsxcv" or $"member_login" === "JakeWharton" or $"member_login" === "haikusw" or $"member_login" === "nathenharvey" or $"member_login" === "jbr" or $"member_login" === "tenderlove" or $"member_login" === "mmp")

td_report.show()

//val topprojects = activeprojectsduration.select("project_id" in td_report("project_id"))

#### Watchers/Starred Events for Top Projects (identified from Developer-Developer Graph)

In order to determine whether developer networks impact repository popularity, the star events are taken from the projects which the top 10 developers with the most collaborators, and are compared with the general trends of the star events of all active projects. To begin, the watchers query shown above is filtered using a where clause in order to display only the repositories which have been identified as the projects worked on by the top 10 developers with the most collaborators (which was found using the page rank algorithm on the developer-developer graph).

In [91]:
'''
Query to return all watchers/starred events for top projects (identified from Developer-Developer graph)

SELECT
  t.repo_id AS repo_id,
  COUNT(*) AS number_watchers,
  t.week_number AS week_number
FROM (
  SELECT
    repo_id,
    created_at,
    WEEK(created_at) AS week_number
  FROM
    [ghtorrent-bq:ght.watchers]
  WHERE
    repo_id = 39095969
    OR repo_id = 17022118
    OR repo_id = 42753525
    OR repo_id = 4230805
    OR repo_id = 43587
    OR repo_id = 11474326
    OR repo_id = 16827100
    OR repo_id = 39780492
    OR repo_id = 6261105
    OR repo_id = 21316774
  ORDER BY
    repo_id,
    week_number ) AS t
GROUP BY
  repo_id,
  week_number
ORDER BY
  repo_id,
  week_number'''

Once imported, the topproj_watchersWeeklyRDD is then cleaned and registered as a SQL table.

In [93]:
%scala

// Define schema
case class topproj_watchersWeekly(                 
   repo_id: Int,
   number_watchers: Int, 
   week_number: Int
  )

// Map data to schema
def topproj_cleanWatchersWeekly(row:Array[String]):topproj_watchersWeekly = {
return topproj_watchersWeekly(
    row(0).toIntOrElse(),
    row(1).toIntOrElse(),
    row(2).toIntOrElse()
  )
}

// Create sql table 
val topproj_watchersweekly = topproj_watchersRDD.map(r => topproj_cleanWatchersWeekly(r)).toDF()

topproj_watchersweekly.show()

// register this data as an SQL table
topproj_watchersweekly.createOrReplaceTempView("topproj_watchersweekly")

#### Project Popularity Predictions

As outlined in the methodology by Antwerp et al. [3], popularity of the repositories will be performed by using clustering - first, the dataset is split into 2 to observe the popularity trends of the projects in the first year of project creation, and four years after project creation. Due to the fact that this analysis is looking at active projects, the Github dataset is split into half - the first 26 weeks (into "dataset_fr") to observe intial project popularity, and the remaining (into "dataset_fr_2") for the time until the one year mark will be used to observe the trends in project popularity. The fr_watchersDF is used since this dataframe was identified to have the full 53 week records of star events (as opposed to incomplete star events for weeks which may be missing). Each of the datasets are split into training and testing set to fit the k-means model and make predictions on the cluster, respectively. A pipeline is created to consist of the VectorAssembler and kmeans function. The k-value is chosen according to the results of the connected components algorithm performed earlier.

In [95]:
%scala

import org.apache.spark.ml.clustering.{KMeans, KMeansModel}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.Pipeline

// load star data
val dataset_fr = fr_watchersDF.select("repo_id", "number_watchers", "week_number").where($"week_number" < 26)
val dataset_fr_2 = fr_watchersDF.select("repo_id", "number_watchers", "week_number").where($"week_number" >= 26)

// split dataset into testing and training set
val split_fr = dataset_fr.randomSplit(Array(0.7, 0.3), seed = 15L)
val train_fr = split_fr(0).cache()
val test_fr = split_fr(1).cache()

val split_fr_2 = dataset_fr_2.randomSplit(Array(0.7, 0.3), seed = 15L)
val train_fr_2 = split_fr_2(0).cache()
val test_fr_2 = split_fr_2(1).cache()

//creating features column
val assembler = new VectorAssembler()
  .setInputCols(Array("repo_id", "number_watchers", "week_number"))
  .setOutputCol("features")

val kmeans = new KMeans()
  .setK(3)
  .setSeed(1L)
  .setFeaturesCol("features")            // setting features column
  .setPredictionCol("prediction")       // setting label column

//creating pipeline
val pipeline = new Pipeline().setStages(Array(assembler,kmeans))

//fitting the model
val kMeansPredictionModel_fr = pipeline.fit(train_fr)
val kMeansPredictionModel_fr_2 = pipeline.fit(train_fr_2)

// make predictions
val predictionResult_fr = kMeansPredictionModel_fr.transform(test_fr)
val predictionResult_fr_2 = kMeansPredictionModel_fr_2.transform(test_fr_2)

The first cell below shows the prediction results of the number of stars for the first 26 weeks of a project. The second cell below shows the prediction results of the number of stars for the remaining time of the year timeframe. When comparing the number_of_watchers v.s. week_number graphs (leftmost grid in the middle row) of the two cells, it is observed that the general trend for projects is that popularity starts of higher and decreases over time.

The k-value here is 3 and as shown in the graphs below, the clusters are as follows:
- green represents the projects with the lowest popularity (have the least number of stars in a given week)
- orange represents the projects with medium popularity (the number of stars in a given week are in the middle)
- blue represents the projects with highest popularity (have the most number of stars in a given week)

In [97]:
%scala

display(predictionResult_fr)

In [98]:
%scala

display(predictionResult_fr_2)

Now a comparison is made with the popularity of the projects which top developers with the most connections to other developers are performed. This begins with splitting the dataset into first 26 weeks, and the remaining time for the second set; as well as creating training and testing data. The k-value is kept the same as the previous k-means model.

In [100]:
%scala

// load star data
val tp_dataset = topproj_watchersweekly.select("*").where($"week_number" < 26)
val tp_dataset_2 = topproj_watchersweekly.select("*").where($"week_number" >= 26)

// split dataset into testing and training set
val tp_split = tp_dataset.randomSplit(Array(0.7, 0.3), seed = 15L)
val tp_train = tp_split(0).cache()
val tp_test = tp_split(1).cache()

val tp_split_2 = tp_dataset_2.randomSplit(Array(0.7, 0.3), seed = 15L)
val tp_train_2 = tp_split_2(0).cache()
val tp_test_2 = tp_split_2(1).cache()

val kmeans_tp = new KMeans()
  .setK(4)
  .setSeed(1L)
  .setFeaturesCol("features")            // setting features column
  .setPredictionCol("prediction")       // setting label column

//creating pipeline
val pipeline_tp = new Pipeline().setStages(Array(assembler,kmeans_tp))

//fitting the model
val kMeansPredictionModel_tp = pipeline_tp.fit(tp_train)
val kMeansPredictionModel_tp_2 = pipeline_tp.fit(tp_train_2)

// make predictions
val predictionResult_tp = kMeansPredictionModel_tp.transform(tp_test)
val predictionResult_tp_2 = kMeansPredictionModel_tp_2.transform(tp_test_2)

The first cell below shows the prediction results of the number of stars for the first 26 weeks of the projects that top developers work on. The second cell below shows the prediction results of the number of stars for the remaining time of the year timeframe. When comparing the number_of_watchers v.s. week_number graphs (leftmost grid in the middle row) of the two cells, it is observed that the trend for projects which have developers who are highly significant (have a large collaborator network), is that the popularity rises with time. This contrasts sharply with the trend of decreasing popularity over time in the general projects.

The k-value here is 4 and as shown in the graphs below, the clusters are as follows:
- green represents the projects with the lowest popularity (have the least number of stars in a given week)
- blue represents the projects which are medium-low in terms of popularity (have the 2nd least number of stars in a given week)
- red represents the projects which are medium-high in terms of popularity (have the 2nd most number of stars in a given week)
- orange represents the projects with highest popularity (have the most number of stars in a given week)

In [102]:
%scala

display(predictionResult_tp)

In [103]:
%scala

display(predictionResult_tp_2)

##### Evaluation - K Means

Silhouette analysis can be used to study the separation distance between the resulting clusters. The silhouette plot is a range from -1 to +1 and is a measure of how close each point in one cluster is to points in the neighboring clusters. It is used to assess parameters such as number of clusters visually. Values close to +1 indicate that the sample is far away from the neighboring clusters.

The silhouette score is calculated for each of the prediction results. As shown in below, the silhouette scores for the fr_watchersDF and topproj_watchersweekly prediction results are around the 0.72-0.82 range, which is relatively high and therefore a good indication that the chosen k-value is sufficient in creating distinct clustering of the datapoints. The cluster centers can be seen in the cells below as well.

In [105]:
%scala

import org.apache.spark.ml.evaluation
import org.apache.spark.ml.evaluation.Evaluator
import org.apache.spark.ml.evaluation.ClusteringEvaluator

// evaluate clustering by computing silhouette score
val evaluator = new ClusteringEvaluator()

val silhouette_fr = evaluator.evaluate(predictionResult_fr)
println("Silhouette with squared euclidean distance (FR): " + silhouette_fr)

println("Cluster Centers (FR): ")
kMeansPredictionModel_fr.stages(1).asInstanceOf[KMeansModel].clusterCenters.foreach(println)

val silhouette_fr_2 = evaluator.evaluate(predictionResult_fr_2)
println("Silhouette with squared euclidean distance (FR2): " + silhouette_fr_2)

println("Cluster Centers (FR2): ")
kMeansPredictionModel_fr_2.stages(1).asInstanceOf[KMeansModel].clusterCenters.foreach(println)

In [106]:
%scala

val silhouette_tp = evaluator.evaluate(predictionResult_tp)
println("Silhouette with squared euclidean distance (TP): " + silhouette_tp)

println("Cluster Centers (TP): ")
kMeansPredictionModel_tp.stages(1).asInstanceOf[KMeansModel].clusterCenters.foreach(println)

val silhouette_tp_2 = evaluator.evaluate(predictionResult_tp_2)
println("Silhouette with squared euclidean distance (TP2): " + silhouette_tp_2)

println("Cluster Centers (TP2): ")
kMeansPredictionModel_tp_2.stages(1).asInstanceOf[KMeansModel].clusterCenters.foreach(println)

#### Summary of Results

The analysis in the modelling section of the report showed that:
- The developer-developer network for active projects on GitHub is connected as evidenced by the connect components algorithm and therefore can be seen as a resilient network
- Developer pairs collaborated together a minimum of 2 times and a maximum of 8 times on different projects
- The maximum number of developers one developer worked with was 165 times
- The general trend of project popularity starts off high but decreases as time goes by
- The projects which are worked on by the top 10 developers who are most connected with other developers in the network, show increasing project popularity trends over time. This contrasts sharply with the general trend where project popularity decreases over time

In conclusion, it can be said that prior developer collaboration on projects does increase project popularity that is measured by the number of stars a project receives, as it allows relationships to foster between developers and to provide a greater understanding of each other's working habits as well as goals.

#### Review / Future Work

Upon review on the current analysis, the following steps were identified as possible areas to review:
- Increasing the size of the datasets - once the data preparation of cleaning and removing invalid data, the final dataset had 13598 developers which is a small portion of the Github data. A larger dataset could be useful in providing concretenidentification of project popularity.
- Including comments and watchers to the popularity measures as the current measure of stars only may not be able to capture the essence of a project's popularity by using a single method.
- As outlined by the Github data study performed by Kalliamvakou et al. [8]:
  * The number of commiters should have been accounted for in order to filter out the personal projects which may have appeared in the list of active projects
  * Should not have relied on the status being merged for the pull request dataset (filter was applied in SQL queries run on BigQuery) as many PRs appear are non-merged even though they are and considered using heuristics instead

#### Bibliography

[1] J. Yang and J. Leskovec, “Patterns of Temporal Variation in Online Media,” in *Proceedings of the Fourth ACM International Conference on Web Search and Data Mining*, New York, NY, USA, 2011, pp. 177–186.

[2] J. Hahn, J. Y. Moon, and C. Zhang, “Impact of Social Ties on Open Source Project Team Formation,” in *Open Source Systems*, 2006, pp. 307–317.

[3] M. V. Antwerp and G. Madey, “The Importance of Social Network Structure in the Open Source Software Developer Community,” in *2010 43rd Hawaii International Conference on System Sciences*, 2010, pp. 1–10.

[4] H. Borges, A. Hora, and M. T. Valente, “Understanding the Factors That Impact the Popularity of GitHub Repositories,” in *2016 IEEE International Conference on Software Maintenance and Evolution (ICSME)*, 2016, pp. 334–344.

[5] H. Borges, A. Hora, and M. T. Valente, “Predicting the Popularity of GitHub Repositories,” in *Proceedings of the The 12th International Conference on Predictive Models and Data Analytics in Software Engineering*, New York, NY, USA, 2016, pp. 9:1–9:10.

[6] G. Gousios, "The GHTorrent dataset and tool suite" in *Proceedings of the 10th Working Conference on Mining Software Repositories (MSR)*, San Francisco, CA, USA, 2013, pp. 233-236

[7] T. Rattenbury, J. M. Hellerstein, J. Heer, C. Carreras, and S. Kandel, Principles of Data Wrangling: Practical Techniques for Data Preparation. O'Reilly Media, 2017.

[8] E. Kalliamvakou, G. Gousios, K. Blincoe, L. Singer, D. M. German, and D. Damian, “The Promises and Perils of Mining GitHub,” in Proceedings of the 11th Working Conference on Mining Software Repositories, New York, NY, USA, 2014, pp. 92–101.